In [1]:
#!/usr/bin/env python
# Created at 2020/5/28
import warnings
import os
import time
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn import preprocessing

warnings.filterwarnings("ignore")

def max_min_scaler(data,cols):
    max_min_scaler = lambda x: (x - np.min(x)) / (np.max(x) - np.min(x) + 0.000001) * 2 - 1
    data[cols] = data[cols].apply(max_min_scaler)

    return data

def log1p_scaler(data,cols):
    log1p_scaler = lambda x: np.log1p(x+0.000001)
    data[cols] = data[cols].apply(log1p_scaler)

    return data


class Drive_control_Dataset(Dataset):
    def __init__(self, data_path, time_step=6, trj_steps=5, rl=False, s_path="./data/common_columns_647.csv"):
        #all_cols = set(pd.read_csv(s_path)["0"])

        self.df = pd.read_csv(data_path)

        all_cols = list(self.df.keys())
        self.df = self.df.loc[:, list(all_cols)]
       # print(self.df.mean())
       # print(self.df.std().to_numpy())
        #print((self.df.std()>0.00000001).to_numpy())


        self.a_t_cols = ['IN_Rail_pressure_feedback',
                         'IN_Gear_ratio',
                         'IN_Coolant_temperature',
                         'ITD_Main_timing_dmnd',
                         'FQD_Chkd_inj_fuel_dmnd',
                         'IN_Pedal_position',
                         'IN_Air_mass_flow',
                         'IN_Vehicle_speed',
                         'T_D_Actual_indicated_torque',
                         'AFC_Air_fuel_ratio',
                         'IN_Manifold_abs_pressure',
                         'IN_Egrh_position',
                         'IN_Vgth_position',
                         'IN_Ext_air_temp',
                         'IN_Throttle_position',
                         'IN_Swirl_position']  # 驾控变量
        self.a_t_cols = [i for i in self.a_t_cols if i in all_cols]
        self.v_t_cols = ['车速(km/h)']  # 或者 'IN_Engine_cycle_speed' 实际车速
        self.v_t_plus_cols = ['理论车速']

        # print(self.df.shape)

        self.o_t_cols = ['CO(ppm)','CO2(%)','THC(ppmC)','NOx(ppm)']

        #self.o_t_cols = ['CO2(%)']

        self.drop_cols = ['时间(s)', 'CO(ppm)', 'CO2(%)', 'THC(ppmC)', 'NOx(ppm)', 'CH4(ppm)', '理论车速',
                          '车速上限', '车速下限', '距离', '累积流量', '颗粒物样气温度', '颗粒物样气进口压力', '颗粒物样气差压',
                          '颗粒物背景气进口温度', '颗粒物背景气进口压力', '主稀释管路空气温度', 'DLS样气瞬时流量', 'DLS背景气瞬时流量',
                          'CPC Count', 'PND1测量值', 'PND2测量值', 'PN Total DF', 'CPC流量', 'SPCS样气压力',
                          'non-CH4', 'N2O', 'CO_MASS(mg)', 'CO2_MASS(mg)', 'THC_MASS(mg)',
                          'NOx_MASS(mg)', 'CH4_MASS(mg)', 'N2O_MASS(mg)', 'Time', 'IN_Engine_cycle_speed']
        self.s_t_cols = list(
            set(all_cols) - set(self.drop_cols + self.v_t_cols + self.v_t_plus_cols + self.o_t_cols + self.a_t_cols))
        self.s_t_cols = [i for i in self.s_t_cols if i in all_cols]

        self.p_cols = [i for i in range(217)]

        cols = self.a_t_cols + self.v_t_cols + self.v_t_plus_cols + self.o_t_cols + self.s_t_cols
        self.df = self.df.loc[:, list(cols)]
        

        drop_cols=[x for i,x in enumerate(self.df.columns) if self.df.iloc[:,i].std()==0]
        self.df=self.df.drop(drop_cols,axis=1) #利用drop方法将含有特定数值的列删除
        all_cols = list(self.df.keys())
        self.a_t_cols = [i for i in self.a_t_cols if i in all_cols]
        self.v_t_cols = ['车速(km/h)']  # 或者 'IN_Engine_cycle_speed' 实际车速
        self.v_t_plus_cols = ['理论车速']
        self.o_t_cols = ['CO(ppm)','CO2(%)','THC(ppmC)','NOx(ppm)']  
        self.s_t_cols = [i for i in self.s_t_cols if i in all_cols]
        cols = self.a_t_cols + self.v_t_cols + self.v_t_plus_cols + self.o_t_cols + self.s_t_cols
        self.df = self.df.loc[:, list(cols)]        
        
        #self.length = self.df.shape[0]
        self.time_step = time_step
        self.trj_steps = trj_steps
        self.steps_every_ulp = 1800
        self.ulp_nums = int(self.df.shape[0] / self.steps_every_ulp)
        self._preprocess()
        self.rl = rl
        self._data_dim()
        self.parse_ulp()


    #数据归一化
    def normalize(self):
        columns = self.df.columns

        self.df = max_min_scaler(self.df, columns)
        self.df = log1p_scaler(self.df, columns)
        self.df = max_min_scaler(self.df, columns)
        #check none data  
        check_null = list(set(list(self.df.isnull().values.reshape(-1))))
        assert (len(check_null) == 1 and False in check_null)


    def parse_ulp(self,ulp_file = "./data/ulp.npy"):
        self.ulps = np.load(ulp_file)


    #数据预处理
    def _preprocess(self):
        self.normalize()
        columns = self.df.columns
        self.ulps_df = {}
        for ulp_index in range(self.ulp_nums):
            data = self.df.iloc[ulp_index*self.steps_every_ulp:(ulp_index+1)*self.steps_every_ulp].to_numpy()
            #
            f_data = np.tile(data[:1,:],(self.time_step-1,1))
            #b_data = np.tile(data[-1:,:],(self.time_step-1,1))

            data = np.concatenate([f_data,data],0)
            self.ulps_df[ulp_index] = pd.DataFrame(data, columns=columns)
            #把下一秒车速赋值给理论车速
            #self.ulps_df[ulp_index].loc[self.time_step:self.time_step+self.steps_every_ulp-1,self.v_t_plus_cols] \
            #    = self.ulps_df[ulp_index].loc[self.time_step+1:self.time_step+self.steps_every_ulp,self.v_t_cols].values
            #print(ulps_df[ulp_index].loc[100:102,['理论车速','车速(km/h)']])


    def _data_dim(self,):
        if self.rl:
            self.dims = {
                "p":len(self.p_cols),
                "s":[self.time_step,len(self.s_t_cols)],
                "v":len(self.v_t_cols),
                "a":len(self.a_t_cols),
                "o":len(self.o_t_cols),
            }
            #self.dims = [len(self.s_t_cols), len(self.v_t_cols), len(self.a_t_cols), len(self.v_t_plus_cols), len(self.s_t_cols)]
        else:

            self.dims = [[2*self.time_step-1,len(self.s_t_cols + self.a_t_cols + self.v_t_cols)], len(self.o_t_cols)]

    def __len__(self):
        if self.rl:
            return self.ulp_nums*(self.steps_every_ulp-1)
        else:
            return self.ulp_nums*self.steps_every_ulp

    #监督学习
    def __getitem__(self, idx):
        if self.rl:
            return self._rl__getitem__(idx)
        else:
            return self._sl__getitem__(idx)

    def _sl__getitem__(self,idx):
        x_cols = self.s_t_cols + self.a_t_cols + self.v_t_cols
        #if self.dims is None:
        #    self.dims = [[2*self.time_step-1,len(x_cols)], len(self.o_t_cols)]
        ulp_index = idx //(self.steps_every_ulp)
        idx = idx % ( self.steps_every_ulp)
        df = self.ulps_df[ulp_index]
        x = df.loc[idx:idx+2*self.time_step-2, x_cols].to_numpy().astype('float')
        y = df.loc[idx+self.time_step-1, self.o_t_cols].to_numpy().astype('float')

        return [torch.from_numpy(i).float() for i in [x,y]]

    def _rl__getitem__(self,idx,trj=False):
        p_cols = self.p_cols
        s_cols = self.s_t_cols
        v_cols = self.v_t_cols
        a_cols = self.a_t_cols
        v_next_cols = self.v_t_plus_cols
        s_next_cols = self.s_t_cols
        o_cols = self.o_t_cols

        ulp_index = idx //(self.steps_every_ulp-1)
        idx = idx % ( self.steps_every_ulp-1)
        df = self.ulps_df[ulp_index]

        p = self.ulps[ulp_index].astype('float')
        
        if trj:
            p =  np.tile(p,(self.trj_steps,1))
            s = np.concatenate([df.loc[idx+i:idx+ i+self.time_step-1, s_cols].to_numpy().astype('float')
                 for i in range(self.trj_steps)],0).reshape(self.trj_steps,self.time_step,-1)
            v = np.concatenate([df.loc[idx+i+self.time_step-1, v_cols].to_numpy().astype('float')
                 for i in range(self.trj_steps)],0).reshape(self.trj_steps,-1)
            a = np.concatenate([df.loc[idx+i+self.time_step-1, a_cols].to_numpy().astype('float')
                 for i in range(self.trj_steps)],0).reshape(self.trj_steps,-1)
            v_next = np.concatenate([df.loc[idx+i+self.time_step-1, v_next_cols].to_numpy().astype('float')
                 for i in range(self.trj_steps)],0).reshape(self.trj_steps,-1)
            s_next = np.concatenate([df.loc[idx+i+self.time_step-1, s_next_cols].to_numpy().astype('float')
                 for i in range(self.trj_steps)],0).reshape(self.trj_steps,-1)
            o = np.concatenate([df.loc[idx+i+self.time_step-1, o_cols].to_numpy().astype('float')
                 for i in range(self.trj_steps)],0).reshape(self.trj_steps,-1)
        else:
            s = df.loc[idx:idx+self.time_step-1, s_cols].to_numpy().astype('float')
            v = df.loc[idx+self.time_step-1, v_cols].to_numpy().astype('float')
            a = df.loc[idx+self.time_step-1, a_cols].to_numpy().astype('float')
            v_next = df.loc[idx+self.time_step - 1, v_next_cols].to_numpy().astype('float')
            s_next = df.loc[idx+self.time_step, s_next_cols].to_numpy().astype('float')
            o = df.loc[idx+self.time_step-1, o_cols].to_numpy().astype('float')
            
            p_n = p
            v_n = df.loc[idx+self.time_step, v_cols].to_numpy().astype('float')
            s_n = df.loc[idx+1:idx+self.time_step, s_cols].to_numpy().astype('float')
            v_next_n = df.loc[idx+self.time_step, v_next_cols].to_numpy().astype('float')
            
            
        #return [torch.from_numpy(i).float() for i in [p,s,v,a,v_next,s_next,o]]
        return [p,s,v,a,v_next,s_next,o,p_n,v_n,v_next_n,s_n]
    

In [2]:
import numpy as np
from tianshou.data import ReplayBuffer,ListReplayBuffer


if __name__ == '__main__':
    
    train_dataset = Drive_control_Dataset(data_path='./data/train.csv',rl=True)
    Buffer_size = len(train_dataset) #  = 1799*2
    buf = ReplayBuffer(size=Buffer_size)
    for i in range(Buffer_size):
        p,s,v,a,v_next,s_next,o,p_n,v_n,v_next_n,s_n = train_dataset[i]
        
        done = (i+1) % 5 == 0
        buf.add(obs={'p': p,'s':s,'v':v,"v_next":v_next,}, 
                act={'a':a,'s_next':s_next,'o':o}, 
                rew=0, 
                done=done,
                obs_next={'p': p_n,'s':s_n,'v':v_n,"v_next":v_next_n,})
        
        if i % 1799 == 0:
            print(i)

0
1799
3598
5397
7196
8995
10794
12593
14392
16191
17990
19789
21588
23387
25186
26985
28784
30583
32382
34181
35980
37779
39578
41377
43176
44975


In [3]:
batch = buf.sample(0)

In [6]:
for i in batch.keys():
    print(type(batch[i]))

<class 'tianshou.data.batch.Batch'>
<class 'tianshou.data.batch.Batch'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'tianshou.data.batch.Batch'>
<class 'tianshou.data.batch.Batch'>
<class 'tianshou.data.batch.Batch'>


In [4]:
import pickle
f = open('train_batch.pkl', 'wb')
pickle.dump(batch, f)

In [5]:
import pickle

f = open('train_batch.pkl', 'rb')
batch,indices = pickle.load(f)

In [6]:
batch

Batch(
    obs: Batch(
             p: array([[7.5e+01, 2.0e+00, 5.0e-01, ..., 6.0e+02, 6.0e+02, 6.0e+02],
                       [7.5e+01, 2.0e+00, 5.0e-01, ..., 6.0e+02, 6.0e+02, 6.0e+02],
                       [7.5e+01, 2.0e+00, 5.0e-01, ..., 6.0e+02, 6.0e+02, 6.0e+02],
                       ...,
                       [7.5e+01, 4.5e+01, 5.0e-01, ..., 5.5e+02, 5.5e+02, 5.5e+02],
                       [7.5e+01, 4.5e+01, 5.0e-01, ..., 5.5e+02, 5.5e+02, 5.5e+02],
                       [7.5e+01, 4.5e+01, 5.0e-01, ..., 5.5e+02, 5.5e+02, 5.5e+02]]),
             s: array([[[ 0.86018077,  0.99999986, -1.        , ..., -1.        ,
                          0.77662985, -1.        ],
                        [ 0.86018077,  0.99999986, -1.        , ..., -1.        ,
                          0.77662985, -1.        ],
                        [ 0.86018077,  0.99999986, -1.        , ..., -1.        ,
                          0.77662985, -1.        ],
                        [ 0.86018077,  0.

In [7]:
batch

Batch(
    act: Batch(
             a: array([[ 0.77556876, -1.        ,  0.60890663, ...,  0.90445048,
                         0.99999986, -1.        ],
                       [ 0.7276507 , -1.        ,  0.59939614, ...,  0.90445048,
                         0.61033303,  0.97586248],
                       [ 0.73552989, -1.        ,  0.60890663, ...,  0.90445048,
                         0.53897358,  0.96698064],
                       ...,
                       [ 0.71986688, -1.        ,  0.98035642, ...,  0.97486708,
                         0.34176502,  0.97467597],
                       [ 0.71639153, -1.        ,  0.98035642, ...,  0.97486708,
                         0.34176502,  0.97467597],
                       [ 0.71986688, -1.        ,  0.98097597, ...,  0.94410705,
                         0.34176502,  0.97467597]]),
             o: array([[0.29228509, 0.15833899, 0.35744343, 0.1267709 ],
                       [0.29343407, 0.15833899, 0.35699894, 0.12419494],
         

In [37]:
import random
random.choice(range(trj_clip_steps))

1

In [41]:
indices.shape[0] // 1799 

26

In [51]:
batch.rew

array([0, 0, 0, ..., 0, 0, 0])

In [122]:
def get_mask_indices(indices,trj_steps,trj_clip_steps):
    #获得不用数据的索引和done
    is_done = True 
    not_done = False
    
    use_steps = trj_steps// trj_clip_steps * trj_clip_steps

    def get_done(index):
        return is_done if (index+1) % 5 ==0 else not_done
    
    mask_indices = []
    done = np.full_like(indices, is_done).astype(bool)
    copy_done = np.array([get_done(i) for i in range(use_steps) ])
    
    for ulp_index in range(indices.shape[0] // trj_steps):

        start_index = random.choice(range(trj_clip_steps-1)) 
        stop_index = use_steps + start_index
        
        start_index = start_index + (ulp_index * trj_steps)
        stop_index = stop_index + (ulp_index * trj_steps)
        
        done[start_index:stop_index] = copy_done
        
        mask_indices += range((ulp_index * trj_steps), start_index)
        mask_indices += range(stop_index, (ulp_index+1) * trj_steps)
        
    return mask_indices,done

In [123]:
mask_indices,done = get_mask_indices(indices,trj_steps,trj_clip_steps)

In [9]:
batch.obs.s

array([[[-1.        ,  0.94412081, -1.        , ...,  0.99999986,
         -1.        ,  0.75337487],
        [-1.        ,  0.94412081, -1.        , ...,  0.99999986,
         -1.        ,  0.75337487],
        [-1.        ,  0.94412081, -1.        , ...,  0.99999986,
         -1.        ,  0.75337487],
        [-1.        ,  0.94412081, -1.        , ...,  0.99999986,
         -1.        ,  0.75337487],
        [-1.        ,  0.94412081, -1.        , ...,  0.99999986,
         -1.        ,  0.75337487],
        [-1.        ,  0.94412081, -1.        , ...,  0.99999986,
         -1.        ,  0.75337487]],

       [[-1.        ,  0.94412081, -1.        , ...,  0.99999986,
         -1.        ,  0.75337487],
        [-1.        ,  0.94412081, -1.        , ...,  0.99999986,
         -1.        ,  0.75337487],
        [-1.        ,  0.94412081, -1.        , ...,  0.99999986,
         -1.        ,  0.75337487],
        [-1.        ,  0.94412081, -1.        , ...,  0.99999986,
         -1.  

In [125]:
batch.donesplit= done

In [128]:
batch.done[:15]

array([ True, False, False, False, False,  True, False, False, False,
       False,  True, False, False, False, False])

In [129]:
def split(self, size: Optional[int] = None,
          shuffle: bool = True, mask_indices: Optional[list] = None,) -> Iterator['Batch']:
    """Split whole data into multiple small batch.
    :param int size: if it is ``None``, it does not split the data batch;
        otherwise it will divide the data batch with the given size.
        Default to ``None``.
    :param bool shuffle: randomly shuffle the entire data batch if it is
        ``True``, otherwise remain in the same. Default to ``True``.
    """
    indices = np.arange(length)
    if mask_indices is not None:
        indices = np.array(list(set(indices) - set(mask_indices)).sort())

    length = indices.shape[0]
    if size is None:
        size = length

    if shuffle:
        np.random.shuffle(indices)

    for idx in np.arange(0, length, size):
        yield self[indices[idx:(idx + size)]]

NameError: name 'BaseExc' is not defined

In [134]:
batch.aa

AttributeError: aa

In [140]:
batch.split(mask_indices=None)

TypeError: split() got an unexpected keyword argument 'mask_indices'

In [1]:
from tianshou.data import Batch

In [2]:
a = Batch()

New


In [3]:
a.split(mask_indices=None)

<generator object Batch.split at 0x7fe08d43ded0>

In [11]:
batch.to_torch()

In [12]:
batch

Batch(
    act: Batch(
             a: tensor([[-1.0000, -1.0000,  0.6011,  ...,  0.9964,  1.0000, -1.0000],
                        [ 0.6346, -1.0000,  0.6011,  ...,  0.9964,  1.0000, -1.0000],
                        [ 0.7348, -1.0000,  0.6011,  ...,  0.9964,  0.6441,  0.9768],
                        ...,
                        [ 0.7067, -1.0000,  0.9522,  ...,  0.9927,  0.4118,  0.9711],
                        [ 0.7078, -1.0000,  0.9522,  ...,  0.9927,  0.4246,  0.9687],
                        [ 0.7056, -1.0000,  0.9528,  ...,  0.9927,  0.4246,  0.9711]],
                       dtype=torch.float64),
             o: tensor([[-0.0655, -0.0794,  0.2286, -0.4059],
                        [-0.0743, -0.0794,  0.2283, -0.5740],
                        [-0.0800, -0.0794,  0.2283, -0.4515],
                        ...,
                        [ 0.1392,  0.3150,  0.2556, -0.1025],
                        [ 0.1369,  0.2938,  0.2545, -0.1091],
                        [ 0.1442,  0.2741,  0.2

In [21]:
a = 'cuda' if torch.cuda.is_available() else 'cpu'

In [22]:
a

'cuda'

In [8]:
import numpy as np
from tianshou.data import Batch
data = Batch(a=4, b=[5, 5], c=Batch(a=4, b=[5, 5]))


In [13]:
data.c = Batch(a=4, b=[5, 6])

In [18]:
data.c.keys()

['a', 'b']